# 表形式のデータ

## データの種類
**原点がない、または絶対的なゼロがないとは**  
何もない状態、つまり「ゼロ」が実際に「何もない」という意味を持つ点のことです。これを小学生にもわかるように例を交えて説明しましょう。

例: 重さ
絶対的なゼロ点の例: 「重さ」を考えてみましょう。もし何も持っていない状態、つまり手が空っぽなら、その重さは「0kg」です。これが絶対的なゼロ点です。ここから重さを測るときは、この「何もない状態」がスタートポイントになります。
例: 温度
絶対的なゼロ点がない例: 一方で、「温度」は絶対的なゼロ点がない場合があります。例えば、摂氏での温度は、0度が「水が凍る温度」を意味しますが、これは「温度が全くない状態」ではありません。実際、温度はマイナスにもなりますし、0度よりも低い温度が存在します。しかし、絶対温度（ケルビン）では、0ケルビンは絶対ゼロと呼ばれ、これは温度が「全くない状態」を意味します。
理解のための簡単なまとめ
絶対的なゼロ点あり: 何もない状態から数え始めることができる（例: 重さが0kgは何もない状態）。
絶対的なゼロ点なし: 0が特定の意味を持つが、それは「何もない状態」ではない（例: 0度は水が凍る温度だが、温度が「全くない」わけではない）。  

**間隔尺度と順序尺度の違い**  
間隔尺度は間隔が一定であり、順序尺度は間隔が一定ではない  

例: 温度計
温度計を例にとって説明しましょう。摂氏での温度計では、10度と20度の差（10度の間隔）は、30度と40度の差（同じく10度の間隔）と等しいです。つまり、温度計上のどの二点を取っても、それらの点間の差は一定です。

重要性
この特性は、データを分析し解釈する際に重要です。例えば、間隔尺度のデータでは、加算や減算が意味を成しますが、乗算や除算は意味を成しません。なぜなら、間隔尺度には絶対ゼロ点がないからです。絶対ゼロ点があれば、乗算や除算が意味を持ちます（これは比例尺度の特性です）。

比較: 順序尺度
これとは対照的に、順序尺度では間の差は一定ではありません。順序尺度では、要素間に順序（ランキング）はありますが、それらの間の具体的な「差」は定義されていないのです。たとえば、映画の評価（☆1つ、☆2つ、☆3つ）では、☆2つと☆3つの間の「質」の差は、☆1つと☆2つの間のそれとは異なるかもしれません。

![データの種類](https://assets.st-note.com/production/uploads/images/48751565/picture_pc_104891899e5cc7b7e4b35e8e23ba3b6d.png)

In [2]:
import torch

## ワインデータをテンソルとして読み込む

In [3]:
import numpy as np
import csv

wine_path = r'C:\Users\shodai\Desktop\PyTorch_Introduction\data\p1ch4\tabular-wine\winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
# print(wineq_numpy)

col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

## Numpyをテンソルに変換

In [4]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape

torch.Size([4898, 12])

## スコアの表現

### 説明変数と目的変数の分離

In [5]:
target = wineq[:, -1]
data = wineq[:, :-1]
print(target)
print(data.shape)

tensor([6., 6., 6.,  ..., 6., 7., 6.])
torch.Size([4898, 11])


### スコアの表現方法　➀整数ラベルとして表現

In [17]:
target1 = target.long()
print(target1)

tensor([6, 6, 6,  ..., 6, 7, 6])


### スコアの表現方法　➁ワンホットエンコードとして表現
※ ネットワークに対してスコアをカテゴリカル変数として扱う時は、スコアをワンホットエンコードに変換しなければいけない

In [22]:
target2 = target.long()
print(target2.shape)

target_onehot = torch.zeros(target2.shape[0], 10)
print(target_onehot.shape)
print(target2.unsqueeze(1).shape)

target_onehot.scatter_(1, target2.unsqueeze(1), 1.0) # scatter_の第二引数はワンホットエンコードするテンソルと同じ次元でないといけない
print(target_onehot.shape)

torch.Size([4898])
torch.Size([4898, 10])
torch.Size([4898, 1])
torch.Size([4898, 10])


## 正規化

In [7]:
data_mean = torch.mean(data, dim=0)
data_std = torch.std(data, dim=0)
# print(data_mean)
# print(data_std)
data_normalized = (data - data_mean) / data_std
# print(data_normalized)

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])


## しきい値

In [9]:
bad_indexes = target <= 3
# print(bad_indexes) # bool値のリスト

bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [10]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [16]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

# NumpyやPyTorchで&は論理演算のandを表す

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [21]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]

# total_sulfur_dataの内、141.1以下であるか示すブール値を返却
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

print(predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum())

torch.Size([4898]) torch.bool tensor(2727)


In [20]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

## 予測の評価

In [37]:
# 予測したものと正解のものの内、共通するインデックスを取得
n_matches = torch.sum(predicted_indexes & actual_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

print(n_matches)
print(n_predicted)
print(n_actual)

print(f'予測したものの内、{n_matches / n_predicted} % が実際に高品質なワインだった')
print(f'正解の内、{n_matches / n_actual} % を特定した')
      


2018
2727
3258
予測したものの内、0.74000733406674 % が実際に高品質なワインだった
正解の内、0.6193984039287906 % を特定した
